In [1]:
import os
import sys
import shelve
from scipy.sparse import csc_array, vstack, coo_array, hstack
import numpy as np

# Append parent dir to system path.
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import rec_sys.cf_algorithms_to_complete as cfa

In [15]:
def estimate_rating(user_id, item_id, user_col, rated_by):
    if str(item_id) not in rated_by:
        print("{} not yet rated by anyone".format(item_id))
        return 0
    
    if str(user_id) not in user_col:
        print("{} not yet rated any item".format(user_id))
        return 0 
    
    # find all users who rated the item
    users = rated_by[str(item_id)]

    # Load all vectors user_col[user] for all users who rated the item
    vectors = [user_col[str(user)] for user in users]
    #vectors.append(user_col[str(user_id)])
    U = csc_array(vstack(vectors).T).toarray()
    rating = cfa.rate_all_items(U, 0, 2)
    print(rating[item_id])
    return rating[item_id]

    

In [16]:
rated_by = shelve.open("../rated_by", flag="r")
user_col = shelve.open("../user_col", flag="r")

user_item_pairs = [
    (828, 11), (2400, 4725), (3765, 1270), (4299, 4020), (5526, 2432),
    (6063, 4525), (7045, 4100), (8160, 6300), (9682, 1212), (10277, 7355)
]


# run the algorithm for all user-item pairs and report the results
results = [estimate_rating(user_id, item_id, user_col, rated_by) for user_id, item_id in user_item_pairs]
print(results)


item_idx: 0, neighbors: [], rating: nan
item_idx: 3, neighbors: [335 218], rating: 5.4
item_idx: 6, neighbors: [59 50], rating: 4.0
item_idx: 25, neighbors: [335  50], rating: 4.1
item_idx: 27, neighbors: [218  50], rating: 4.9
item_idx: 29, neighbors: [335 218], rating: 5.4
item_idx: 30, neighbors: [348  59], rating: 5.2
item_idx: 33, neighbors: [335  50], rating: 5.1
item_idx: 35, neighbors: [335  50], rating: 5.1
item_idx: 41, neighbors: [218  50], rating: 3.9
item_idx: 42, neighbors: [335  50], rating: 4.1
item_idx: 44, neighbors: [335  50], rating: 5.1
item_idx: 45, neighbors: [ 56 335], rating: 4.6
item_idx: 46, neighbors: [335  50], rating: 5.1
item_idx: 49, neighbors: [335  50], rating: 4.7
item_idx: 50, neighbors: [218  50], rating: 4.4
item_idx: 51, neighbors: [335  50], rating: 5.1
item_idx: 53, neighbors: [335 218], rating: 2.3
item_idx: 54, neighbors: [335 218], rating: 3.3
item_idx: 57, neighbors: [335  50], rating: 5.1
item_idx: 59, neighbors: [265  56], rating: 3.4
item

In [14]:
import psutil
import os
import time

def measure_memory_usage(user_id, item_id, user_col, rated_by):
    # Start tracking memory usage
    process = psutil.Process(os.getpid())
    start_mem = process.memory_info().rss / (1024 * 1024)  # Convert to MB
    
    # Estimate rating and track peak memory usage
    rating = estimate_rating(user_id, item_id, user_col, rated_by)
    max_mem = start_mem
    for _ in range(10):  # Monitor memory usage over a short period
        current_mem = process.memory_info().rss / (1024 * 1024)  # In MB
        max_mem = max(max_mem, current_mem)
        time.sleep(0.1)  # Sleep for a short time to allow peak tracking
    
    return rating, max_mem

# Load your data structures (shelve or otherwise)
rated_by = shelve.open("../rated_by", flag="r")
user_col = shelve.open("../user_col", flag="r")

# User-item pairs for testing
user_item_pairs = [
    (828, 11), (2400, 4725), (3765, 1270), (4299, 4020), (5526, 2432), (6063, 4525)
]

# Evaluate memory usage for each user-item pair
results = []
for user_id, item_id in user_item_pairs:
    rating, max_memory = measure_memory_usage(user_id, item_id, user_col, rated_by)
    results.append((user_id, item_id, rating, max_memory))
    print(f"User {user_id}, Item {item_id} - Rating: {rating}, Max Memory Usage: {max_memory:.2f} MB")

# Close the shelve files
rated_by.close()
user_col.close()

# Display all results
for user_id, item_id, rating, max_memory in results:
    print(f"User {user_id}, Item {item_id}: Estimated rating = {rating}, Max memory usage = {max_memory:.2f} MB")


item_idx: 0, neighbors: [], rating: nan
item_idx: 3, neighbors: [ 78  26 204 401   6 236 219 168 325 297 418 265  63 356 178 348  59  56
 335 218], rating: 4.7
item_idx: 6, neighbors: [ 44 205 264 241 233   9 110   3  26   6 236 219 168 297 418  71 356 348
  59  50], rating: 3.9
item_idx: 25, neighbors: [233 414 110   3  78 402 204 401   6 236 219 168 325 418 265  63 356 178
 335  50], rating: 4.3
item_idx: 27, neighbors: [ 78  26 402 204   6 236 219 168 325 297 265  63 356 178 348  59  56 335
 218  50], rating: 4.9
item_idx: 29, neighbors: [110   3  78  26 402 204 401   6 168 325 297 418 265  63 356 178 348  56
 335 218], rating: 4.9
item_idx: 30, neighbors: [403 369  32 307 352 109 227  39 291  93 340 239   9 110  78  26   6 265
 348  59], rating: 4.4
item_idx: 33, neighbors: [311 414   3  26 402 204 401   6 168 325 297 418 265  63 356 178 348  56
 335  50], rating: 4.3
item_idx: 35, neighbors: [153 241   3 401   6 236 219 168 325 297 418  71 265  63 356 178 348  56
 335  50], rating